<a href="https://colab.research.google.com/github/wendelfsilva/aerlytix-lease/blob/main/Aerlytix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install numpy-financial -q

In [33]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import random as rd
import pandas as pd
import numpy as np
import numpy_financial as npf

In [34]:
data = {
  "pricing": {
      "economic_closing_date": datetime(2020, 4, 1),
      "net_price": 35000000,
      "num_samples": 1000
  },
  "lease": {
      "start_date": datetime(2020, 7, 15),
      "end_date": datetime(2026, 7, 15),
      "monthly_rent_dollars": 150000,
      "monthly_mr_rate_dollars": 135000
  },
  "maintenance": {
      "check_interval_months": 60,
      "initial_used_life_months": 30,
      "check_cost_min_dollars": 2000000,
      "check_cost_max_dollars": 5000000
  },
  "residual_value": {
      "appraised_aircraft_value": 35000000,
      "as_of": datetime(2020, 7, 15),
      "depreciation_rate": 0.05
  }
}

In [35]:
# function to generate the period range between two dates
def generate_period(lease: dict):
    start = lease['start_date'].replace(day=1)
    end = lease['end_date'].replace(day=1)
    period = pd.period_range(start=start, end=end, freq='M')
    # period[0] = min(lease['start_date'], start)
    return period

In [36]:
# function to generate random maintenance value
def generate_maintenance_value(maintenance: dict):
    return rd.randrange(maintenance['check_cost_min_dollars'], maintenance['check_cost_max_dollars'])

In [37]:
# function to calculate the difference between two dates
def date_diff_in_days(date1, date2=None):
    if not date2:
        date2 = date1.replace(day=1) + relativedelta(months=1, days=-1)

    if date2 > date1:
        return (date2.date() - date1.date()).days
    else:
        return (date1.date() - date2.date()).days

In [38]:
def generate_cash_flow(
    date: datetime,
    value: float,
    mr_value: float,
    description: str = 'rent + MR',
  ):
 
  # if date was not the first date I would calculate the date diff
  if date.day != 1:  
      value_per_day = value / 30
      value = value_per_day * date_diff_in_days(date)

      description += ' (partial month)'

  return {
      'date': date,
      'cash_flow': value,    
      'mr_balance': mr_value,
      'description': description
  }

In [39]:
# creating object to store all cash flows
cash_flows = []

# accessing object lease
pricing = data['pricing']

# creating flow to ECD
obj = {
    'date': pricing['economic_closing_date'],
    'cash_flow': -pricing['net_price'],
    'mr_balance': None,
    'description': 'Net Price on ECD'
}
cash_flows.append(obj)

In [40]:
# accessing object lease & maitenance
lease = data['lease']
maintenance = data['maintenance']

# mr balance acumulate
mr_balance_sum = 0
used_life_sum = maintenance['initial_used_life_months']

# looping period
for p in generate_period(lease=lease):
  
    # building all infos  
    date = p.start_time
    description = 'rent + MR'
    
    # check if is the first date  
    if date.month == lease['start_date'].month and \
       date.year == lease['start_date'].year:
        date = max(date, lease['start_date'])

    # increase used life  
    mr_balance_sum += lease['monthly_mr_rate_dollars']
    used_life_sum += 1
    
    # check if have to do the maintenance  
    if used_life_sum == maintenance['check_interval_months']:    
        used_life_sum = 0
        description += ' - maintenance check cost'

        maintenance_value = generate_maintenance_value(maintenance=maintenance)      
        mr_balance_sum -= maintenance_value
        if mr_balance_sum < 0:
            mr_balance_sum = 0      
    
    obj = generate_cash_flow(    
        date=date,
        value=lease['monthly_rent_dollars'],
        mr_value=mr_balance_sum,
        description=description
    )
    cash_flows.append(obj)

In [41]:
# accessing models
lease = data['lease']
residual = data['residual_value']

years_from_valuation = (lease['end_date'] - residual['as_of']).days / 365
depreciation_rate = residual['depreciation_rate'] * years_from_valuation
depreciation_value = residual['appraised_aircraft_value'] * depreciation_rate

residual_value = residual['appraised_aircraft_value'] - depreciation_value

# creating flow to ECD
obj = {
    'date': lease['end_date'],
    'cash_flow': residual_value + mr_balance_sum,
    'mr_balance': 0,
    'description': 'Aircraft sale (%.2f) + MR balance leftover (%.2f)' % (residual_value, mr_balance_sum)
}
cash_flows.append(obj)

In [42]:
df = pd.DataFrame(cash_flows).round()
df

,date,cash_flow,mr_balance,description
0,2020-04-01,-35000000.0,NaN,Net Price on ECD
1,2020-07-15,80000.0,135000.0,rent + MR (partial month)
2,2020-08-01,150000.0,270000.0,rent + MR
3,2020-09-01,150000.0,405000.0,rent + MR
4,2020-10-01,150000.0,540000.0,rent + MR
...,...,...,...,...
70,2026-04-01,150000.0,5551427.0,rent + MR
71,2026-05-01,150000.0,5686427.0,rent + MR
72,2026-06-01,150000.0,5821427.0,rent + MR
73,2026-07-01,150000.0,5956427.0,rent + MR


In [43]:
# calculating IRR (%)

expected_irr = npf.irr(df['cash_flow']) * 100

expected_irr = round(expected_irr, 2)

expected_irr

0.26

In [44]:
# build final result

final_result = {
    'cash_flow': [],
    'mr_balance': [],
    'pricing': {}
}

for row in df.iloc[1:].itertuples():
  final_result['cash_flow'].append([row.date.strftime('%Y-%m-%d'), row.cash_flow])
  final_result['mr_balance'].append([row.date.strftime('%Y-%m-%d'), row.mr_balance])

final_result['pricing']['expected_irr'] = round(npf.irr(df['cash_flow']) * 100, 2)

final_result

{'cash_flow': [['2020-07-15', 80000.0],
  ['2020-08-01', 150000.0],
  ['2020-09-01', 150000.0],
  ['2020-10-01', 150000.0],
  ['2020-11-01', 150000.0],
  ['2020-12-01', 150000.0],
  ['2021-01-01', 150000.0],
  ['2021-02-01', 150000.0],
  ['2021-03-01', 150000.0],
  ['2021-04-01', 150000.0],
  ['2021-05-01', 150000.0],
  ['2021-06-01', 150000.0],
  ['2021-07-01', 150000.0],
  ['2021-08-01', 150000.0],
  ['2021-09-01', 150000.0],
  ['2021-10-01', 150000.0],
  ['2021-11-01', 150000.0],
  ['2021-12-01', 150000.0],
  ['2022-01-01', 150000.0],
  ['2022-02-01', 150000.0],
  ['2022-03-01', 150000.0],
  ['2022-04-01', 150000.0],
  ['2022-05-01', 150000.0],
  ['2022-06-01', 150000.0],
  ['2022-07-01', 150000.0],
  ['2022-08-01', 150000.0],
  ['2022-09-01', 150000.0],
  ['2022-10-01', 150000.0],
  ['2022-11-01', 150000.0],
  ['2022-12-01', 150000.0],
  ['2023-01-01', 150000.0],
  ['2023-02-01', 150000.0],
  ['2023-03-01', 150000.0],
  ['2023-04-01', 150000.0],
  ['2023-05-01', 150000.0],
  ['2023

In [45]:
# generating json file

import json 

with open('result.json', 'w') as file:
  file.write(json.dumps(final_result))

In [46]:
# view the data distribution by year

init_df = df.iloc[:1].groupby(by=[df['date'].dt.year]).sum()

cff_df = df.iloc[1:].groupby(by=[df['date'].dt.year]).sum()

irr_df = init_df.append(cff_df).reset_index()

irr_df[['date', 'cash_flow']]

,date,cash_flow
0,2020,-35000000.0
1,2020,830000.0
2,2021,1800000.0
3,2022,1800000.0
4,2023,1800000.0
5,2024,1800000.0
6,2025,1800000.0
7,2026,31501632.0
